In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import hydra
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import xarray as xr

In [3]:
from crims2s.dataset import S2SDataset, TransformedDataset
from crims2s.training.model.emos import NormalEMOSModel, LinearModel, NormalCubeNormalEMOS
from crims2s.training.model.util import PytorchMultiplexer
from crims2s.transform import AddLatLonFeature, AddBiweeklyDimTransform, CompositeTransform
from crims2s.util import ECMWF_FORECASTS, collate_with_xarray

In [5]:
with hydra.initialize_config_module('crims2s.training.conf'):
    cfg = hydra.compose('config', overrides=['experiment/dataset=ncep'])

In [6]:
t = hydra.utils.instantiate(cfg.experiment.transform)

In [7]:
t

CompositeTransform([<crims2s.transform.AddGeographyFeatures object at 0x2accdaf40c10>, <crims2s.transform.AddBiweeklyDimTransform object at 0x2accdafbb850>, <crims2s.transform.LinearModelAdapter object at 0x2accdafbb970>, <crims2s.transform.AddMetadata object at 0x2accdafbb940>, <crims2s.transform.ExampleToPytorch object at 0x2accdafbb700>, <crims2s.transform.CubeRootTP object at 0x2accdafbb730>, <crims2s.transform.AddLatLonFeature object at 0x2accdafbbbe0>])

In [ ]:
t.transforms = t.transforms[:-3]

In [8]:
d = TransformedDataset(S2SDataset(cfg.experiment.dataset.dataset_dir), t)

In [14]:
[k for k in d[0].keys() if k.startswith('model_parameters')]

['model_parameters_t2m_mu',
 'model_parameters_t2m_sigma',
 'model_parameters_tp_alpha',
 'model_parameters_tp_beta',
 'model_parameters_tp_mu',
 'model_parameters_tp_sigma',
 'model_parameters_tp_cube_root_mu',
 'model_parameters_tp_cube_root_sigma']

In [15]:
d[0]['model_parameters_t2m_mu'].shape

torch.Size([2, 121, 240])

In [11]:
d[0]['features_features'].shape

torch.Size([6, 121, 240, 11, 17])

In [16]:
loader = torch.utils.data.DataLoader(d, batch_size=4, collate_fn=collate_with_xarray)

In [17]:
one_batch = next(iter(loader))

In [18]:
one_batch['eccc_parameters_tp_mu'].shape

torch.Size([4, 2, 121, 240])

In [ ]:
next(iter(loader))